# Day 19: Aplenty

## Part 1

The Elves of Gear Island are thankful for your help and send you on your way. They even have a hang glider that someone stole from Desert Island; since you're already going that direction, it would help them a lot if you would use it to get down there and return it to them.

As you reach the bottom of the relentless avalanche of machine parts, you discover that they're already forming a formidable heap. Don't worry, though - a group of Elves is already here organizing the parts, and they have a system.

To start, each part is rated in each of four categories:

- `x`: Extremely cool looking
- `m`: Musical (it makes a noise when you hit it)
- `a`: Aerodynamic
- `s`: Shiny

Then, each part is sent through a series of workflows that will ultimately accept or reject the part. Each workflow has a name and contains a list of rules; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)

Consider the workflow `ex{x>10:one,m<20:two,a>30:R,A}`. This workflow is named ex and contains four rules. If workflow ex were considering a specific part, it would perform the following steps in order:

- Rule "`x>10:one`": If the part's x is more than 10, send the part to the workflow named one.
- Rule "`m<20:two`": Otherwise, if the part's m is less than 20, send the part to the workflow named two.
- Rule "`a>30:R`": Otherwise, if the part's a is more than 30, the part is immediately rejected (R).
- Rule "`A`": Otherwise, because no other rules matched the part, the part is immediately accepted (A).

If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is accepted (sent to `A`) or rejected (sent to `R`), the part immediately stops any further processing.

The system works, but it's not keeping up with the torrent of weird metal shapes. The Elves ask if you can help sort a few parts and give you the list of workflows and some part ratings (your puzzle input). For example:

```
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
```

The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named in. In this example, the five listed parts go through the following workflows:

- `{x=787,m=2655,a=1222,s=2876}`: `in` -> `qqz` -> `qs` -> `lnx` -> `A`
- `{x=1679,m=44,a=2067,s=496}`: `in` -> `px` -> `rfg` -> `gd` -> `R`
- `{x=2036,m=264,a=79,s=2244}`: `in` -> `qqz` -> `hdj` -> `pv` -> `A`
- `{x=2461,m=1339,a=466,s=291}`: `in` -> `px` -> `qkq` -> `crn` -> `R`
- `{x=2127,m=1623,a=2188,s=1013}`: `in` -> `px` -> `rfg` -> `A`

Ultimately, three parts are accepted. Adding up the `x`, `m`, `a`, and `s` rating for each of the accepted parts gives `7540` for the part with `x=787`, `4623` for the part with `x=2036`, and `6951` for the part with `x=2127`. Adding all of the ratings for all of the accepted parts gives the sum total of `19114`.

Sort through all of the parts you've been given; what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?



In [59]:
from __future__ import annotations
from dataclasses import dataclass


@dataclass
class Part:
    x: int
    m: int
    a: int
    s: int

    @staticmethod
    def from_raw(raw: str) -> Part:
        raw = f"dict({raw[1:-1]})"
        data = eval(raw)
        return Part(**data)


Part.from_raw("{x=787,m=2655,a=1222,s=2876}")

Part(x=787, m=2655, a=1222, s=2876)

In [60]:
class Rule:
    def __init__(self, prop: str, operator: str, value: int, acceptance: str):
        self.prop = prop
        self.operator = operator
        self._operator_func = int.__lt__ if operator == "<" else int.__gt__
        self.value = value
        self.acceptance = acceptance

    def __repr__(self) -> str:
        return f"Rule({self.prop}{self.operator}{self.value}:{self.acceptance})"
    
    def test(self, part: Part) -> bool:
        return self._operator_func(part.__dict__[self.prop], self.value)

    @staticmethod
    def from_raw(raw: str) -> Rule:
        if ":" not in raw:
            return AcceptRule(raw)
        split = raw.split(":")
        condition_raw = split[0]
        return Rule(
            prop=condition_raw[0],
            operator=condition_raw[1], 
            value=int(condition_raw[2:]), 
            acceptance=split[1],
        )


class AcceptRule(Rule):
    def __init__(self, acceptance: str):
        self.acceptance = acceptance

    def __repr__(self) -> str:
        return f"AcceptRule({self.acceptance})"

    def test(self, part: Part) -> bool:
        return True



test_rule = Rule.from_raw("x<5:accept")
print(test_rule)
print(test_rule.test(Part(4, 0, 0, 0)))
print(test_rule.test(Part(6, 0, 0, 0)))

Rule(x<5:accept)
True
False


In [61]:
class Workflow:
    def __init__(self, name: str, rules: list[Rule]):
        self.name = name
        self.rules = rules

    def __repr__(self) -> str:
        return f"Workflow({self.name}, {self.rules})"

    def test(self, part: Part) -> str:
        for rule in self.rules:
            if rule.test(part):
                return rule.acceptance

    @staticmethod
    def from_raw(raw: str) -> Workflow:
        name, rules_raw = raw.rstrip("}").split("{")
        rules = [Rule.from_raw(rule_raw) for rule_raw in rules_raw.split(",")]
        return Workflow(name, rules)
    

test_workflow = Workflow.from_raw("px{a<2006:qkq,m>2090:A,rfg}")
print(test_workflow)
print(test_workflow.test(Part(0, 0, 2005, 0)))
print(test_workflow.test(Part(0, 2091, 2007, 0)))
print(test_workflow.test(Part(0, 0, 2007, 0)))

Workflow(px, [Rule(a<2006:qkq), Rule(m>2090:A), AcceptRule(rfg)])
qkq
A
rfg


In [62]:
test_data_raw = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""


def parse_data(data_raw):
    workflows_raw, parts_raw = data_raw.split("\n\n")

    workflows = dict()
    for workflow_raw in workflows_raw.splitlines():
        workflow = Workflow.from_raw(workflow_raw)
        workflows[workflow.name] = workflow
    parts = [Part.from_raw(part_raw) for part_raw in parts_raw.splitlines()]

    return workflows, parts


test_data = parse_data(test_data_raw)
test_data

({'px': Workflow(px, [Rule(a<2006:qkq), Rule(m>2090:A), AcceptRule(rfg)]),
  'pv': Workflow(pv, [Rule(a>1716:R), AcceptRule(A)]),
  'lnx': Workflow(lnx, [Rule(m>1548:A), AcceptRule(A)]),
  'rfg': Workflow(rfg, [Rule(s<537:gd), Rule(x>2440:R), AcceptRule(A)]),
  'qs': Workflow(qs, [Rule(s>3448:A), AcceptRule(lnx)]),
  'qkq': Workflow(qkq, [Rule(x<1416:A), AcceptRule(crn)]),
  'crn': Workflow(crn, [Rule(x>2662:A), AcceptRule(R)]),
  'in': Workflow(in, [Rule(s<1351:px), AcceptRule(qqz)]),
  'qqz': Workflow(qqz, [Rule(s>2770:qs), Rule(m<1801:hdj), AcceptRule(R)]),
  'gd': Workflow(gd, [Rule(a>3333:R), AcceptRule(R)]),
  'hdj': Workflow(hdj, [Rule(m>838:A), AcceptRule(pv)])},
 [Part(x=787, m=2655, a=1222, s=2876),
  Part(x=1679, m=44, a=2067, s=496),
  Part(x=2036, m=264, a=79, s=2244),
  Part(x=2461, m=1339, a=466, s=291),
  Part(x=2127, m=1623, a=2188, s=1013)])

In [63]:
def is_accepted(part, workflows):
    current_workflow_name = "in"
    while current_workflow_name not in {"A", "R"}:
        current_workflow = workflows[current_workflow_name]
        current_workflow_name = current_workflow.test(part)
    return current_workflow_name == "A"


def part1(data):
    workflows, parts = data
    result = 0
    for part in parts:
        if is_accepted(part, workflows):
            result += part.x + part.m + part.a + part.s
    return result


part1(test_data)

19114

In [64]:
with open("input.txt") as f:
    data = parse_data(f.read())

data[0]["in"], data[1][:3]

(Workflow(in, [Rule(s<1496:qfr), Rule(m<1482:tj), AcceptRule(nqp)]),
 [Part(x=4, m=211, a=430, s=167),
  Part(x=469, m=1450, a=3037, s=1084),
  Part(x=90, m=929, a=457, s=661)])

In [65]:
part1(data)

446517

## Part 2

Even with your help, the sorting process still isn't fast enough.

One of the Elves comes up with a new plan: rather than sort parts individually through all of these workflows, maybe you can figure out in advance which combinations of ratings will be accepted or rejected.

Each of the four ratings (`x`, `m`, `a`, `s`) can have an integer value ranging from a minimum of `1` to a maximum of `4000`. Of all possible distinct combinations of ratings, your job is to figure out which ones will be accepted.

In the above example, there are `167409079868000` distinct combinations of ratings that will be accepted.

Consider only your list of workflows; the list of part ratings that the Elves wanted you to sort is no longer relevant. How many distinct combinations of ratings will be accepted by the Elves' workflows?

In [70]:
import math
import numpy as np


def count_accepted(workflows):
    def helper(workflow_name, rule_idx, bounds={prop: (1, 4000) for prop in "xmas"}):
        OPERATORS = "><"

        if workflow_name in ["A", "R"]:
            return math.prod([upper - lower + 1 for lower, upper in bounds.values()]) if workflow_name == "A" else 0
        
        workflow = workflows[workflow_name]
        rule = workflow.rules[rule_idx]

        if isinstance(rule, AcceptRule):
            return helper(rule.acceptance, 0, bounds)
            
        result = 0
        lower_bound = rule.value - 1 if rule.operator == "<" else rule.value
        upper_bound = rule.value if rule.operator == "<" else rule.value + 1
        new_base_bounds = (lower_bound, upper_bound)

        for axis in [0, 1]:
            if bounds[rule.prop][0] < new_base_bounds[axis] < bounds[rule.prop][1]:
                new_bounds = bounds.copy()
                new_bounds[rule.prop] = np.array([bounds[rule.prop][axis], new_base_bounds[axis]])[[axis, 1 - axis]]
                new_workflow_name, new_rule_idx = (workflow_name, rule_idx + 1) if rule.operator == OPERATORS[axis] else (rule.acceptance, 0)
                result += helper(new_workflow_name, new_rule_idx, new_bounds)

        return result
    
    return helper("in", 0)


def part2(data):
    return count_accepted(data[0])


part2(test_data)

167409079868000

In [71]:
part2(data)

130090458884662